In [1]:
import sys
sys.path.append('../')

from model import DecisionTree
from rule import RuleExtractor

import os
import numpy as np
import warnings
import pandas as pd
import matplotlib.pyplot as plt

np.set_printoptions(suppress=True)

warnings.filterwarnings(action='ignore')

import seaborn as sns
sns.set(font="Malgun Gothic", rc={"axes.unicode_minus":False}, style='white')

In [2]:
# sage, srgnn 중 선택
embed_model = 'srgnn'
filename = {'sage' : 'user_embedding_sage_sid_6to6_weighted.npy', 'srgnn' : 'srgnn_user_hybrid_6to6_weighted.npy'}
data_path = "../data/"

pivot_df = pd.read_csv(os.path.join(data_path, 'pivot_genre2_data.csv'), index_col=0)
embedding = np.load(os.path.join(data_path, filename[embed_model]))

model = DecisionTree(pivot_df, embedding)
model.dt = model.make_dt(min_samples_leaf=25, min_impurity_decrease=0.0007)

In [4]:
k=15
len_or_list = []
save_dir = f'../output/{embed_model}_rule'
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

rule_extractor = RuleExtractor(model)
for i in range(k):
    rule = rule_extractor.extract_rule(segment_num=i)
    rule_extractor.save_rule(rule, f'../output/srgnn_rule/srgnn_seg_rule_extract{i}.txt')

In [79]:
for i in range(15):
    with open(f'../output/srgnn_rule/srgnn_seg_rule_extract{i}.txt', 'r') as f:
        full_text = f.read()
    
    not_rule_text = ''
    or_list = full_text.split('[OR]')


    for or_idx, or_rule in enumerate(or_list):
        and_list = or_rule.split('[AND]')
        
        not_list = []
        normal_list = []
        
        for and_rule in and_list:
            and_rule = and_rule.strip()

            if '<=' in and_rule:
                front, back = and_rule.split('<=')
                back = round(float(back)*60)
                not_list.append(f'{front} > {back} ')

            elif '>' in and_rule:
                front, back = and_rule.split('>')
                back = round(float(back)*60)
                normal_list.append(f'{front} > {back} ')

        cnt = 0

        for normal_rule in normal_list:
            if cnt != 0:
                not_rule_text += '[AND] '
            
            not_rule_text += normal_rule
            cnt+=1
        
        for not_rule in not_list:
            if cnt != 0:
                not_rule_text += '[NOT] '
            not_rule_text += not_rule

        if len(not_rule_text) != 0 and or_idx != len(or_list)-1:
            not_rule_text += '[OR]\n'

    with open(f'../output/final_srgnn_not_rule/srgnn_seg_rule_extract{i}.txt', 'w') as f:
        f.writelines(not_rule_text)

In [81]:
for i in range(15):
    with open(f'../output/final_srgnn_not_rule/srgnn_seg_rule_extract{i}.txt', 'r') as f:
        full_text = f.read()

    full_text = full_text.replace('\n', '')
    rule_text = ''
    or_list = full_text.split('[OR]')

    for or_idx, or_rule in enumerate(or_list):
        rule = or_rule.split('[NOT]')[0]
        if len(rule.strip()) == 0:
            continue
        rule_text += rule
        if or_idx != len(or_list)-1:
            rule_text += '[OR]\n'
    
    with open(f'../output/final_srgnn_rule/srgnn_seg_rule_extract{i}.txt', 'w') as f:
        f.writelines(rule_text)